# Summary Generation Round B

In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os, json, re

## Knowledge Base

In [8]:
from knowledge_base import KnowledgeBase
kb = KnowledgeBase()

In [9]:
kb.keywords_metadata()

Failed to get data: response.status_code=422, response=<Response [422]>


In [15]:
"""
/search_papers
基于文本内容搜索相关论文片段。
[
    {
        'id': 454846909877948052, 
        'distance': 0.6174427270889282,
        'entity': {
            'paper_id': '658254cf939a5f4082bc95a5',
            'paper_title': 'Point Cloud Part Editing: Segmentation, Generation, Assembly, and Selection',
            'chunk_id': 3,
            'chunk_text': '# Loss Functions\nWe adopt the loss function introduced in Wasserstein GAN ...',
            'original_filename': 'Conf_Paper_Meta_Data_AAAI2024_with_whole_text.db',
            'year': 2024
        }
    },
]
"""
query = "损失函数"
chunks = kb.search_papers(query, top_k=1)
chunks[0]

{'id': 454846909877948052,
 'distance': 0.6174427270889282,
 'entity': {'paper_id': '658254cf939a5f4082bc95a5',
  'paper_title': 'Point Cloud Part Editing: Segmentation, Generation, Assembly, and Selection',
  'chunk_id': 3,
  'chunk_text': '# Loss Functions\nWe adopt the loss function introduced in Wasserstein GAN (Arjovsky, Chintala, and Bottou 2017) with gradient penalty (Gulrajani et al. 2017). Network $E$ ${\\vec{\\mathfrak{z}}},\\,G_{p i},i=1...n,$ ,$F_{p i},i=1...n$ , and $F$ need training. The losses are given as:  \n\n$$\n\\begin{array}{c c}{\\displaystyle{\\mathcal{L}_{G}=-\\alpha\\ast\\frac{1}{n}\\sum_{i=1}^{n}\\mathbb{E}_{z\\sim Z}[F_{p i}(G_{p i}(z))]}}\\\\ {\\displaystyle}\\\\ {\\displaystyle}\\\\ {\\displaystyle}\\\\ {\\displaystyle{\\mathcal{L}_{F_{p}}=\\frac{1}{n}\\sum_{i=1}^{n}(\\mathbb{E}_{z\\sim Z}[F(\\bigcup_{i=1}^{n}G_{p i}(z))]}}\\\\ {\\displaystyle}\\\\ {\\displaystyle{\\mathcal{L}_{F_{p}}=\\frac{1}{n}\\sum_{i=1}^{n}(\\mathbb{E}_{z\\sim Z}[F_{p i}(G_{p i}(z))]-\

In [16]:
"""
/query_whole_text_by_id
通过论文ID查询论文全文。
"""
paper_id = '658254cf939a5f4082bc95a5'
paper_text = kb.query_whole_text_by_id(paper_id)

In [43]:
"""
/query_by_paper_id
根据论文ID查询相关的论文片段。
"""
paper_id = '658254cf939a5f4082bc95a5'
chunks = kb.query_by_paper_id(paper_id, top_k=1000)
chunks = sorted(chunks, key=lambda x: ['chunk_id'])
chunks_text = "\n".join([chunk['chunk_text'] for chunk in chunks])

In [57]:
"""
/query_by_title_contain
搜索标题中包含特定关键词的论文片段。
严格匹配，区分大小写。
"""
chunks = kb.query_by_title_contain("Loss")
chunks[0]

{'id': 454845485404913716,
 'paper_id': '6350bc6d90e50fcafdecf289',
 'paper_title': 'RankT5: Fine-Tuning T5 for Text Ranking with Ranking Losses.',
 'chunk_id': 0,
 'chunk_text': '# RankT5: Fine-Tuning T5 for Text Ranking with Ranking Losses\nHonglei Zhuang, Zhen Qin, Rolf Jagerman, Kai Hui, Ji Ma, Jing Lu, Jianmo Ni, Xuanhui Wang and Michael Berdersky Google Research  \n\n{hlz,zhenqin,jagerman,kaihuibj,maji,ljwinnie,jianmon, xuanhui,bemike}@google.com\n\n# Abstract\nRecently, substantial progress has been made in text ranking based on pretrained language models such as BERT. However, there are limited studies on how to leverage more powerful sequence-to-sequence models such as T5. Existing attempts usually formulate text ranking as classification and rely on postprocessing to obtain a ranked list. In this paper, we propose RankT5 and study two T5-based ranking model structures, an encoder-decoder and an encoder-only one, so that they not only can directly output ranking scores for eac

In [66]:
"""
/query_by_title_like
搜索标题中包含特定关键词的论文片段。
模糊匹配，不区分大小写。
返回值: 嵌套的论文片段信息列表。 外层列表对应每个相似标题，内层列表是与该相似标题相关的论文片段信息列表 (结构同 /search_papers)。
"""
chunks = kb.query_by_title_like("loSs", top_k=1)
len(chunks[0])

7

In [62]:
"""
/query_by_chunk_contain
搜索论文片段内容中包含特定关键词的论文片段。
"""
chunks = kb.query_by_chunk_contain("LoSS", top_k=1)
chunks[0]


{'id': 454845515631169384,
 'paper_id': '6432998690e50fcafd467a44',
 'paper_title': 'Cross-Modal Retrieval with Partially Mismatched Pairs',
 'chunk_id': 7,
 'chunk_text': '# $D$ . Comparison With Rectifying Method\nIn this section, we compare our RCL with the most related method NCR [43] to investigate the effectiveness and efficiency of the proposed learning paradigm. First, NCR requires simultaneously training two individual cross-modal models with GMM in a co-teaching manner, which will take a relatively high computational cost. In contrast, our method does not introduce extra training costs into the original cross-modal method, thus embracing higher efficiency. Second, we conduct some comparisons with NCR in Table V. From the experiments, one could see that both NCR and our RCL achieve comparable retrieval performance in low mismatching rates (e.g., 0.2 and 0.4). However, the performance of NCR will fast degrade with high mismatching rates (e.g., 0.6 and 0.8) because NCR cannot co

In [70]:
"""
/query_keywords_by_id
通过论文ID查询论文关键词。
返回值: 关键词列表 (字符串列表) 或 None (如果未找到)。
"""
paper_id = '658254cf939a5f4082bc95a5'
paper_keywords = kb.query_keywords_by_id(paper_id)
paper_keywords

['Mesh Segmentation', 'Point Clouds']

In [69]:
"""
/query_by_keyword
通过关键词查询论文ID和标题。
返回值: 论文ID和标题列表。 每个元素是一个元组 (paper_id, title)。
"""
results = kb.query_by_keyword("Transfer Learning")
results

[['64c33dc33fda6d7f069586ec',
  'Multi-Source Domain Adaptation Through Dataset Dictionary Learning in Wasserstein Space'],
 ['652d36a9939a5f4082496263',
  'DTBS: Dual-Teacher Bi-Directional Self-Training for Domain Adaptation in Nighttime Semantic Segmentation'],
 ['622577a75aee126c0f008dd0',
  'Continual Few-shot Relation Learning Via Embedding Space Regularization and Data Augmentation'],
 ['60dfc18591e01129379b371c',
  'Combining Feature and Instance Attribution to Detect Artifacts'],
 ['623004305aee126c0f9b37d6',
  'Continual Prompt Tuning for Dialog State Tracking'],
 ['6232a74d5aee126c0fe13f03',
  'ConTinTin: Continual Learning from Task Instructions'],
 ['6075694591e0110f6fe68296',
  'Meta-Learning for Fast Cross-Lingual Adaptation in Dependency Parsing'],
 ['62393e7e5aee126c0f125f36',
  'Leveraging Expert Guided Adversarial Augmentation for Improving Generalization in Named Entity Recognition'],
 ['62393e7e5aee126c0f125ef5',
  'Continual Sequence Generation with Adaptive Compo